
# Lemniscata de Penin (∞̸) — Validação por Simulação

Este notebook demonstra o operador **∞̸ (infinito sob trilhos)** na prática:
- Implementação mínima do operador como classe Python (`LemniscataPenin`).
- Simulação de evolução com **novidade (N)**, **integridade (I)** e **novidade inadmissível (iN)**.
- Comparação com um baseline **sem trilhos**.
- Geração de gráficos, logs (CSV) e resumo (Markdown).


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from pathlib import Path
import textwrap

# Exibir gráficos inline (ao abrir no Jupyter)
%matplotlib inline



## Operador ∞̸ (infinito sob trilhos)

Formalização programática mínima:

\[ P = \infty\̸ (E + N - iN), \quad \text{com} \; iN = (1 - I) \cdot N, \; I \in [0, 1] \]


In [ ]:

@dataclass
class LemniscataPenin:
    """
    Operador 'infinito sob trilhos' (∞̸).
    P = ∞̸(E + N − iN), com iN = (1 − I) * N e I ∈ [0,1].
    """
    def __call__(self, E: float, N: float, I: float) -> float:
        I = float(np.clip(I, 0.0, 1.0))
        N = float(max(0.0, N))
        iN = (1.0 - I) * N
        P = E + N - iN
        return max(E, P)  # garante não regredir o estado

lemniscata = LemniscataPenin()



## Simulação

- **N** é proposta de novidade com distribuição normal truncada em [0, 0.30].  
- **I** (integridade) é derivada de um risco que cresce com N (com ruído).  
- **Sem trilhos** aceita tudo e sofre instabilidade quando o risco sobe.  
- **Com ∞̸** só passa a parte íntegra.


In [ ]:

def propose_novelty(rng: np.random.Generator) -> float:
    N = rng.normal(0.06, 0.04)
    return float(np.clip(N, 0.0, 0.30))

def compute_integrity_from_risk(N: float, rng: np.random.Generator) -> tuple[float, float]:
    risk = np.clip(1.2 * N + rng.normal(0.0, 0.05), 0.0, 1.0)
    I = 1.0 - risk
    return float(np.clip(I, 0.0, 1.0)), float(risk)

rng = np.random.default_rng(42)
T = 300
E_lem = 0.40
E_raw = 0.40
rows = []

for t in range(1, T + 1):
    N = propose_novelty(rng)
    I, risk = compute_integrity_from_risk(N, rng)

    # Lemniscata (∞̸)
    P = lemniscata(E_lem, N, I)
    accepted = P > E_lem
    E_lem_next = P if accepted else E_lem

    # Baseline sem trilhos
    E_raw_next = E_raw + N
    if risk > 0.70:
        E_raw_next = max(0.0, E_raw_next - risk * 0.8)
    elif risk > 0.50:
        E_raw_next = max(0.0, E_raw_next - risk * 0.3)

    E_lem = float(np.clip(E_lem_next, 0.0, 1.2))
    E_raw = float(np.clip(E_raw_next, 0.0, 1.2))
    iN = (1.0 - I) * N

    rows.append({
        "iter": t,
        "N": N,
        "I": I,
        "risk": risk,
        "iN": iN,
        "P_lem": P,
        "accepted_lem": int(accepted),
        "E_after": E_lem,
        "E_raw": E_raw,
    })

df = pd.DataFrame(rows)
df.head()



## Gráficos


In [ ]:

plt.figure()
plt.plot(df["iter"], df["E_after"], label="Com ∞̸ (nos trilhos)")
plt.plot(df["iter"], df["E_raw"], label="Sem trilhos (aceita tudo)")
plt.xlabel("Iteração")
plt.ylabel("Eficiência (E)")
plt.title("Evolução da Eficiência: Lemniscata ∞̸ vs Sem Trilhos")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

plt.figure()
plt.plot(df["iter"], df["I"], label="Integridade (I)")
plt.xlabel("Iteração")
plt.ylabel("Integridade (I)")
plt.title("Integridade por Iteração")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

plt.figure()
plt.plot(df["iter"], df["N"], label="Novidade (N)")
plt.plot(df["iter"], df["iN"], label="Novidade inadmissível (iN)")
plt.xlabel("Iteração")
plt.ylabel("Valores")
plt.title("Novidade (N) vs Novidade Inadmissível (iN)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

plt.figure()
plt.plot(df["iter"], df["accepted_lem"], label="Mutação aceita (∞̸)")
plt.xlabel("Iteração")
plt.ylabel("Aceita (1) / Rejeita (0)")
plt.title("Decisões de Aceitação sob ∞̸")
plt.legend()
plt.tight_layout()
plt.show()



## Artefatos (logs e resumo)


In [ ]:

out_dir = Path("/mnt/data")
out_dir.mkdir(parents=True, exist_ok=True)

csv_path = out_dir / "lemniscata_validacao_logs.csv"
df.to_csv(csv_path, index=False)

summary = f"""
# Relatório-Resumo — Validação da Lemniscata de Penin (∞̸)

**Iterações:** {len(df)}

**Eficiência final (∞̸):** {df['E_after'].iloc[-1]:.4f}
**Eficiência final (sem trilhos):** {df['E_raw'].iloc[-1]:.4f}
**Aceitação média (∞̸):** {df['accepted_lem'].mean():.2f}

Notas:
- O operador ∞̸ aceita apenas a parcela íntegra da novidade (N − iN) e mantém evolução sustentável.
- O baseline sem trilhos, ao aceitar tudo, sofre instabilidades quando o risco é alto.
"""

md_path = out_dir / "lemniscata_validacao_relatorio.md"
md_path.write_text(textwrap.dedent(summary), encoding="utf-8")

csv_path, md_path
